In [ ]:
import pickle
import requests
import os

API_KEY_VISUAL_CROSSING = "YOUR_API_KEY_HERE"

In [20]:
def ppj(data):
    """
    Takes a JSON object (dictionary or string) and prints it in a pretty format.
    
    Args:
    data (dict or str): The JSON object to be pretty printed.
    
    Returns:
    None
    """
    if isinstance(data, str):
        # If the input is a JSON string, parse it into a dictionary
        data = json.loads(data)
    
    # Pretty print the JSON with indentation
    print(json.dumps(data, indent=4, sort_keys=True))

In [ ]:


def get_single_visual_crossing_record(datetime_str, location= "New York, NY", api_key=API_KEY_VISUAL_CROSSING):
    """
    Queries Visual Crossing for a single weather record at the specified time.
    """
    
    # The datetime string is now passed directly into the API path
    date_time_path = datetime_str
    
    base_url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{location}/{date_time_path}"
    
    params = {
        'unitGroup': 'us',  # Set your preferred units ('us' or 'metric')
        'include': 'current',   # Critical filter to return only the single record for the time
        'key': api_key,
        'contentType': 'json'
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Check for HTTP errors
        data = response.json()
        #ppj(data)
        
        # The single record is returned in the 'currentConditions' field
        if 'currentConditions' in data:
            record = data['currentConditions']
            
            # Extract the required fields
            return {
                'datetime_local': datetime_str,
                'location': data.get('resolvedAddress', location),
                'temperature F': record.get('temp'),
                'humidity PCT': record.get('humidity'),
                'wind speed mph': record.get('windspeed'),
                'feels like F': record.get('feelslike'),
                'conditions': record.get('conditions').lower()
            }
        else:
            print(f"Error: API response missing 'currentConditions' for {datetime_str}")
            return None

    except requests.exceptions.RequestException as e:
        print(f"API Request failed for {datetime_str}: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred for {datetime_str}: {e}")
        return None



In [ ]:
#Using a weather API:
years = range(1999, 2000)
for year in years:
    races = pickle.load(open(f'race_details/race_details_for_year_{year}.pickle', 'rb'))
    for race in races:
        start_time = races[race]["startDateTime"]
        weather = get_single_visual_crossing_record(start_time)
        races[race]["visual_crossing_weather"] = weather
    fn_out = f'race_details_with_visual_crossing_weather/race_details_for_year_with_visual_crossing_weather_{year}.pickle'
    if os.path.dirname(fn_out):
        os.makedirs(os.path.dirname(fn_out), exist_ok=True)
    pickle.dump(races, open(fn_out, 'wb'))
    

    


In [ ]:
year=1999
ppj(len((pickle.load(open(f'race_details_with_visual_crossing_weather/race_details_for_year_with_visual_crossing_weather_{year}.pickle', 'rb')))))
print(len(pickle.load(open(f'race_details/race_details_for_year_{year}.pickle', 'rb'))))